# General exercises

## Motorcycle problem

A front-end problem faced with pure python (3.8.6)

Related link: https://youtu.be/aouDQ8caJYg?t=225

Custom config (for better data visualization):
 - Time intervals: from 8:00 a.m. to 8:30 a.m.
 - Amount of motorcyclists per time interval: 3

### Multiprocessing version

_Taxis instead of motorcycles_

Basic setup

In [1]:
import logging

ADDRESS = '127.0.0.1', 8000
PASSWORD = b'secret password'

print_process = logging.getLogger('multiprocessing')
print_process.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('[%(process)d] - [%(processName)s]: %(message)s'))
print_process.addHandler(handler)

Server side

In [2]:
import datetime
import multiprocessing
import time
from multiprocessing import connection


class Response:

    def __init__(self, success, *, message=None, data=None):
        self.success = success
        self.message = message
        self.data = data


class Config:

    def __init__(self, taxis_amount=8, start_minutes=480, end_minutes=1200, interval=30, debug=False):
        self.taxis_amount = taxis_amount
        self.start_minutes = start_minutes
        self.end_minutes = end_minutes
        self.interval = interval
        self.debug = debug


class Debug:

    def __init__(self, debug):
        self.debug = debug
    
    def print_debug(self, user, message):
        if self.debug:
            print_process.info('[%(user)s]: %(message)s', {'user': user, 'message': message})


class TaxisGroup(Debug):

    EMPTY = 0
    AVAILABLE = 1
    CONTRACTED = 2

    def __init__(self, taxis_amount, minutes, debug):
        super().__init__(debug)
        date = datetime.datetime(1, 1, 1) + datetime.timedelta(minutes=minutes)
        self.hour = date.hour
        self.minutes = date.minute
        self.taxis_amount = multiprocessing.Queue(taxis_amount)
        for n in range(taxis_amount):
            self.taxis_amount.put(f'Taxi #{n} ({self})')
        self.manager = multiprocessing.Manager()
        self.users = self.manager.dict()

    def __repr__(self):
        return str(self)

    def __str__(self):
        return f'{self.hour:0>2}:{self.minutes:0>2}'

    def __is_empty(self):
        return self.taxis_amount.empty()

    def count(self):
        return self.taxis_amount.qsize()

    def toggle(self, user):
        status = self.check_status(user, 'TOGGLE')
        if status == self.AVAILABLE:
            data = self.taxis_amount.get()
            self.users[user] = data
            self.print_debug(user, f'users: {self.users}')
            return f'Get {data}'
        elif status == self.CONTRACTED:
            data = self.users.pop(user)
            self.taxis_amount.put(data)
            return f'Released {data}'
        return 'Invalid action'

    def check_status(self, user, debug_message):
        self.print_debug(user, f'check_status [{debug_message}]: {self.users}')
        if user in self.users:
            return self.CONTRACTED
        elif self.__is_empty():
            return self.EMPTY
        return self.AVAILABLE


class Server(Debug):

    def __init__(self, address, authkey, config):
        super().__init__(config.debug)
        self.taxi_schedules = []
        self.address = address
        self.authkey = authkey
        self.__setup(config)

    def __setup(self, config):
        self.taxi_schedules.extend(TaxisGroup(config.taxis_amount, minutes, self.debug) for minutes in range(config.start_minutes, config.end_minutes, config.interval))

    def __handle_connection(self, conn, user):
        print_process.info('New user connected: %(user_id)s', {'user_id': user})
        while True:
            try:
                request = conn.recv()
            except EOFError:
                break
            self.print_debug(user, f'request: {request}')
            response = self.__generate_response(request, user)
            conn.send(response.__dict__)
        print_process.info('End connection with: %(user)s', {'user': user})

    def __generate_response(self, request, user):
        if not isinstance(request, dict):
            return Response(False, message='Bad request')
        elif 'method' not in request:
            return Response(False, message='Missing method')
        elif request['method'] not in ('GET', 'POST'):
            return Response(False, message='Method not allowed')
        elif request['method'] == 'POST':
            if 'index' not in request:
                return Response(False, message='Miss index')
            elif not isinstance(request['index'], int):
                return Response(False, message='Index must be an integer')
            return self.post(user, request['index'])
        return self.get(user)

    def start(self):
        print_process.info('Start server on %(host)s:%(port)s', {'host': self.address[0], 'port': self.address[1]})
        with connection.Listener(self.address, authkey=self.authkey) as listener:
            while True:
                try:
                    with listener.accept() as conn:
                        multiprocessing.Process(target=self.__handle_connection, args=(conn, listener.last_accepted)).start()
                except KeyboardInterrupt:
                    print_process.info('Server stopped')

    def get(self, user):
        """Handle user GET request"""
        data = [
            {
                'count': taxi_group.count(),
                'status': taxi_group.check_status(user, 'GET'),
            }
            for taxi_group in self.taxi_schedules
        ]
        return Response(True, data=data)

    def post(self, user, index):
        """Handle user POST request (click)"""
        try:
            self.taxi_schedules[index].toggle(user)
        except IndexError:
            return Response(False, message='Invalid index')
        return Response(True)


server = Server(ADDRESS, PASSWORD, Config(taxis_amount=3, start_minutes=480, end_minutes=540))
server_process = multiprocessing.Process(target=server.start)
server_process.start()

[282133] - [Process-3]: Start server on 127.0.0.1:8000
[282144] - [Process-3:1]: New user connected: ('127.0.0.1', 38562)
[282149] - [Process-3:2]: New user connected: ('127.0.0.1', 38564)
[282153] - [Process-3:3]: New user connected: ('127.0.0.1', 38566)
[282165] - [Process-3:4]: New user connected: ('127.0.0.1', 38574)
[282149] - [Process-3:2]: End connection with: ('127.0.0.1', 38564)
[282153] - [Process-3:3]: End connection with: ('127.0.0.1', 38566)
[282165] - [Process-3:4]: End connection with: ('127.0.0.1', 38574)
[282144] - [Process-3:1]: End connection with: ('127.0.0.1', 38562)
[282133] - [Process-3]: Server stopped


User side

In [3]:
import itertools
import multiprocessing
import time
from multiprocessing import connection


class History:

    def __init__(self, action, data, result):
        self.action = action
        self.data = data
        if self.action == Action.verbose_action(Action.GET):
            self.result = self.render_interface(result)
        else:
            self.result = result
        self.time = time.time()

    def __repr__(self):
        return str(self)

    def __str__(self):
        return str(self.__dict__)

    @staticmethod
    def render_interface(result):
        status = {
            0: 'Red',  # EMPTY
            1: 'White',  # AVAILABLE
            2: 'Green',  # CONTRACTED
        }
        for taxis_groups in result['data']:
            taxis_groups['status'] = status[taxis_groups['status']]
        return result


class Action:

    SLEEP = 0
    GET = 1
    POST = 2

    def __init__(self, action, data=None):
        self.action = action
        self.data = data or {}

    def __call__(self, user, conn):
        if self.action == self.SLEEP:
            result = time.sleep(self.data)
        else:
            result = self.__request(conn)
        user.history.put(History(self.verbose_action(self.action), self.data, result))

    def __request(self, conn):
        conn.send(dict({'method': self.verbose_action(self.action)}, **self.data))
        return conn.recv()

    @staticmethod
    def verbose_action(action):
        if action == Action.SLEEP:
            return 'SLEEP'
        elif action == Action.GET:
            return 'GET'
        return 'POST'


class User:

    def __init__(self, address, authkey, behavior):
        self.address = address
        self.authkey = authkey
        self.behavior = behavior
        self.history = multiprocessing.Queue()

    def connect(self):
        with connection.Client(self.address, authkey=self.authkey) as conn:
            time.sleep(1)
            for action in self.behavior:
                action(self, conn)


# Sleep actions applied only for better visualization of the results
user_1_behavior = (
    Action(Action.GET),
    Action(Action.POST, {'index': 0}),
    Action(Action.GET),
    Action(Action.SLEEP, 4),
    Action(Action.POST, {'index': 1}),
    Action(Action.GET),
    Action(Action.POST, {'index': 0}),
    Action(Action.GET),
)
user_2_behavior = (
    Action(Action.SLEEP, 1),
    Action(Action.GET),
    Action(Action.POST, {'index': 0}),
    Action(Action.GET),
    Action(Action.POST, {'index': 1}),
    Action(Action.GET),
)
user_3_behavior = (
    Action(Action.SLEEP, 2),
    Action(Action.GET),
    Action(Action.POST, {'index': 0}),
    Action(Action.GET),
)
user_4_behavior = (
    Action(Action.SLEEP, 3),
    Action(Action.GET),
)
users = (
    User(ADDRESS, PASSWORD, user_1_behavior),
    User(ADDRESS, PASSWORD, user_2_behavior),
    User(ADDRESS, PASSWORD, user_3_behavior),
    User(ADDRESS, PASSWORD, user_4_behavior),
)
processes = [multiprocessing.Process(target=user.connect) for user in users]
for process in processes:
    process.start()
for process in processes:
    process.join()
print_process.info('User simulation ended!')

[282108] - [MainProcess]: User simulation ended!


Process the user data

In [4]:
# Pandas and IPython (third party libraries) only for data visualization
import pandas as pd
from IPython.display import display

users_history = []
for n, user in enumerate(users):
    user_history = []
    while not user.history.empty():
        user_history.append(dict({'user_id': n}, **user.history.get().__dict__))
    users_history.extend(user_history)

sorted_by_time = sorted(users_history, key=lambda x: x['time'])
parsed_list = []
for item in sorted_by_time:
    user_id = item['user_id']
    data = [[item['time']]]
    data.extend([['', '', ''] for _ in range(len(users))])
    try:
        converted_data = '|'.join(f'{result["status"]},{result["count"]}' for result in item['result']['data'])
    except (KeyError, TypeError):
        converted_data = ''
    data[1 + user_id] = [
        item['action'],
        str(item['data'] or ''),
        converted_data,
    ]
    parsed_list.append([column for user_data in data for column in user_data])

columns = pd.MultiIndex.from_tuples([
    ['Time', ''],
    *itertools.product((f'User {n}' for n in range(len(users))), ('Action', 'Data', 'Response')),
])
df = pd.DataFrame(parsed_list, columns=columns)

# Improve Jupyter Lab visualization
method_columns = tuple((f'User {n}', 'Action') for n in range(len(users)))
def highlight_methods(s):
    serie = pd.Series(data=s)
    for n, row in enumerate(serie):
        if serie._index[n] in method_columns:
            if 'POST' == row:
                return ['background-color: #00F5D4' for x in serie]
            elif 'GET' == row:
                return ['background-color: #F15BB5' for x in serie]
    return
stylized_df = df.style.apply(highlight_methods, axis=1)

with pd.option_context('display.max_colwidth', None):
    display(stylized_df)

### Threading version

In [5]:
import collections
import datetime
import logging
import queue
import threading
import time

logger = logging.getLogger('Thread')
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setFormatter(logging.Formatter('[%(threadName)s]: %(message)s'))
logger.addHandler(handler)

AVAILABLE = 0
CONTRACTED = 1
EMPTY = 2


class MotorcyclistsGroup:

    def __init__(self, amount, minutes):
        date = datetime.datetime(1, 1, 1) + datetime.timedelta(minutes=minutes)
        self.hour = date.hour
        self.minutes = date.minute

        self.availables = queue.Queue(amount)
        for n in range(amount):
            self.availables.put(f'Motorcyclist #{n} ({self})')

        self.users = {}
        self.users_lock = threading.Lock()

    def __repr__(self):
        return str(self)

    def __str__(self):
        return f'{self.hour:0>2}:{self.minutes:0>2}'
        
    def is_empty(self):
        return self.availables.empty()

    def count(self):
        return self.availables.qsize()

    def toggle(self):
        thread_name = threading.current_thread().name
        with self.users_lock:
            status = self.check_status()
            if status == CONTRACTED:
                data = self.users.pop(thread_name)
                self.availables.put(data)
                return f'Released {data}'
            elif status == AVAILABLE:
                data = self.availables.get()
                self.users[thread_name] = data
                return f'Get {data}'
            return 'Invalid action'

    def check_status(self):
        if threading.current_thread().name in self.users:
            return CONTRACTED
        elif self.is_empty():
            return EMPTY
        return AVAILABLE


class User:

    def __init__(self):
        self.contracted_schedules = collections.defaultdict(lambda: None)

    def click(self, schedule):
        data = schedule.toggle()
        self.contracted_schedules[schedule] = data
        return data


def check(schedule):
    status = {
        AVAILABLE: 'White',
        CONTRACTED: 'Green',
        EMPTY: 'Red',
    }
    logger.info('Status (%(schedule)s): %(status)s', {'schedule': schedule, 'status': status[schedule.check_status()]})
    logger.info('Amount (%(schedule)s): %(count)s' , {'schedule': schedule, 'count': schedule.count()})


def user_1_actions(motorcyclists):
    user = User()

    check(motorcyclists[0])
    check(motorcyclists[1])

    logger.info(user.click(motorcyclists[0]))
    check(motorcyclists[0])
    check(motorcyclists[1])

    time.sleep(4)

    logger.info(user.click(motorcyclists[1]))
    check(motorcyclists[0])
    check(motorcyclists[1])
    
    logger.info(user.click(motorcyclists[0]))
    check(motorcyclists[0])
    check(motorcyclists[1])


def user_2_actions(motorcyclists):
    user = User()

    time.sleep(1)

    check(motorcyclists[0])
    check(motorcyclists[1])

    logger.info(user.click(motorcyclists[0]))
    check(motorcyclists[0])
    check(motorcyclists[1])

    logger.info(user.click(motorcyclists[1]))
    check(motorcyclists[0])
    check(motorcyclists[1])


def user_3_actions(motorcyclists):
    user = User()

    time.sleep(2)

    check(motorcyclists[0])
    check(motorcyclists[1])
    
    logger.info(user.click(motorcyclists[0]))
    check(motorcyclists[0])
    check(motorcyclists[1])


def user_4_actions(motorcyclists):
    user = User()

    time.sleep(3)

    check(motorcyclists[0])
    check(motorcyclists[1])


motorciclists_amount = 3  # Default 8
intervals = 2  # Default 25
motorcyclists = [MotorcyclistsGroup(motorciclists_amount, 480 + 30 * n) for n in range(intervals)]
logger.info(motorcyclists)
user_1 = threading.Thread(name='User 1', target=user_1_actions, args=(motorcyclists,))
user_2 = threading.Thread(name='User 2', target=user_2_actions, args=(motorcyclists,))
user_3 = threading.Thread(name='User 3', target=user_3_actions, args=(motorcyclists,))
user_4 = threading.Thread(name='User 4', target=user_4_actions, args=(motorcyclists,))
users = [user_1, user_2, user_3, user_4]
for user in users:
    user.start()
for user in users:
    user.join()

[MainThread]: [08:00, 08:30]
[User 1]: Status (08:00): White
[User 1]: Amount (08:00): 3
[User 1]: Status (08:30): White
[User 1]: Amount (08:30): 3
[User 1]: Get Motorcyclist #0 (08:00)
[User 1]: Status (08:00): Green
[User 1]: Amount (08:00): 2
[User 1]: Status (08:30): White
[User 1]: Amount (08:30): 3
[User 2]: Status (08:00): White
[User 2]: Amount (08:00): 2
[User 2]: Status (08:30): White
[User 2]: Amount (08:30): 3
[User 2]: Get Motorcyclist #1 (08:00)
[User 2]: Status (08:00): Green
[User 2]: Amount (08:00): 1
[User 2]: Status (08:30): White
[User 2]: Amount (08:30): 3
[User 2]: Get Motorcyclist #0 (08:30)
[User 2]: Status (08:00): Green
[User 2]: Amount (08:00): 1
[User 2]: Status (08:30): Green
[User 2]: Amount (08:30): 2
[User 3]: Status (08:00): White
[User 3]: Amount (08:00): 1
[User 3]: Status (08:30): White
[User 3]: Amount (08:30): 2
[User 3]: Get Motorcyclist #2 (08:00)
[User 3]: Status (08:00): Green
[User 3]: Amount (08:00): 0
[User 3]: Status (08:30): White
[User 3